In [12]:
from datetime import datetime
import automlShortForecast.util as util
import pandas as pd
#from automlShortForecast.automlShortForecast import automlShortForecast
from sklearn import linear_model
from matplotlib import pyplot as plt
import pickle

In [13]:
#!pip install automlShortForecast

In [14]:
x_df=pd.read_csv('x.csv',index_col=0)
y_df=pd.read_csv('y.csv',index_col=0)

In [15]:
x_df.head(10)

,i.set,X_basic.forecast_time,X_basic.horizon,X_basic.time,X_basic.hour,EC.nwp_time,EC.dist,EC.ws,EC.wd,EC.rho,EC.pres,EC.tmp,GFS.nwp_time,GFS.dist,GFS.ws,GFS.wd,GFS.rho,GFS.pres,GFS.tmp
0,0,2018-07-01 00:00:00,0,2018-07-01 00:00:00,0,2018-06-30 12:00:00,12.0,5.584500,221.91003,1.152237,995.48000,27.827000,2018-06-30 12:00:00,12.0,5.6870,218.53998,1.136570,987.24,29.450
1,0,2018-07-01 00:00:00,1,2018-07-01 01:00:00,1,2018-06-30 12:00:00,13.0,5.054566,217.49670,1.148961,995.42000,28.667000,2018-06-30 12:00:00,13.0,6.0588,220.27002,1.133212,987.41,30.399
2,0,2018-07-01 00:00:00,2,2018-07-01 02:00:00,2,2018-06-30 12:00:00,14.0,4.524633,213.08331,1.145703,995.36000,29.507000,2018-06-30 12:00:00,14.0,5.9294,221.65002,1.130534,987.39,31.112
3,0,2018-07-01 00:00:00,3,2018-07-01 03:00:00,3,2018-06-30 12:00:00,15.0,3.994700,208.66998,1.142463,995.30000,30.347000,2018-06-30 12:00:00,15.0,5.5855,220.58002,1.127922,987.45,31.835
4,0,2018-07-01 00:00:00,4,2018-07-01 04:00:00,4,2018-06-30 12:00:00,16.0,4.593200,209.38666,1.142332,994.96330,30.279000,2018-06-30 12:00:00,16.0,5.5985,218.66998,1.126699,987.22,32.095
5,0,2018-07-01 00:00:00,5,2018-07-01 05:00:00,5,2018-06-30 12:00:00,17.0,5.191700,210.10333,1.142202,994.62665,30.211000,2018-06-30 12:00:00,17.0,5.7533,212.68000,1.125234,986.76,32.350
6,0,2018-07-01 00:00:00,6,2018-07-01 06:00:00,6,2018-06-30 12:00:00,18.0,5.790200,210.82000,1.142071,994.29000,30.143000,2018-06-30 12:00:00,18.0,6.2740,209.72998,1.124392,986.18,32.399
7,0,2018-07-01 00:00:00,7,2018-07-01 07:00:00,7,2018-06-30 12:00:00,19.0,5.687467,209.89667,1.142320,993.95000,29.973333,2018-06-30 12:00:00,19.0,6.9883,209.25000,1.124169,985.52,32.255
8,0,2018-07-01 00:00:00,8,2018-07-01 08:00:00,8,2018-06-30 12:00:00,20.0,5.584733,208.97333,1.142568,993.61000,29.803667,2018-06-30 12:00:00,20.0,7.2806,210.69000,1.125011,984.95,31.850
9,0,2018-07-01 00:00:00,9,2018-07-01 09:00:00,9,2018-06-30 12:00:00,21.0,5.482000,208.04999,1.142817,993.27000,29.634000,2018-06-30 12:00:00,21.0,7.2318,210.35999,1.127775,984.78,31.050


In [16]:
def feature_clean(x_df, y_df=None, cleaning_x=True):
    if cleaning_x:
        if isinstance(x_df, pd.DataFrame):
            is_valid = x_df.notnull().all(axis=1)
            print(89, is_valid.shape)

        else:
            is_valid = x_df.notnull()
#             print(92, is_valid.shape)
    else:
        is_valid = pd.Series(np.ones(len(x_df), dtype=bool), index=x_df.index)
    if y_df is not None:
        if isinstance(y_df, pd.DataFrame):
            is_valid = is_valid & y_df.notnull().all(axis=1)
        else:
            is_valid = is_valid & y_df.notnull()
    x_clean = x_df.loc[is_valid]
#     print(99, x_clean.shape)
#     print(100, is_valid)
#     print(101, )
    if y_df is not None:
        y_clean = y_df.loc[is_valid]
        return x_clean, y_clean, is_valid
    else:
        return x_clean, is_valid

In [17]:
def data_prepare(x_df):
    x_df = util.polar_to_cart(x_df)
    x_df = util.add_feature_wdcut(x_df, n_sector=8, one_hot_encoding=True)
    x_df = util.add_feature_rho_crossed(x_df, '.ws')
    return x_df

In [18]:
def _1st_stacking_feature(nwp):
    input_feature = [nwp + '.ws_rho1', nwp + '.ws_rho2', nwp + '.ws_angles', nwp + '.ws',
                     nwp + '.wd', nwp + '.rho', nwp + '.dist', nwp + '.u', nwp + '.v']
    input_feature.extend(['{}.wd_cut_s{}'.format(nwp, n) for n in range(8)])
    output_feature = nwp + '.pre_ws'
    return input_feature, output_feature

In [19]:
def LinearModel():
    return linear_model.LinearRegression()

# PreprocessingLinearModel:
    original data preprocessing and training
    input:
        x_df:feature frame
        y_df:label frame
        pickle_name:model weight name
     return: 
        trained model 

In [20]:
def PreprocessingLinearModel(x_df, y_df,pickle_name='Linear_model.pkl'):
    x_df, y_df, _ = feature_clean(x_df, y_df, cleaning_x=True)
    print(x_df)
    x_df = data_prepare(x_df)
    input_feature1, output_feature1 = _1st_stacking_feature('EC')
    input_feature2, output_feature2 = _1st_stacking_feature('GFS')
    input_feature=input_feature1+input_feature2
    model=LinearModel()
    fitted_model=model.fit(x_df[input_feature], y_df)
    y_pred=fitted_model.predict(x_df[input_feature])
    with open(pickle_name, 'wb') as file:
        pickle.dump(fitted_model, file)
    plt.figure(figsize=(20,10))
    plt.plot(y_pred)
    plt.plot(y_df)
    plt.show()
    return fitted_model
    
    

# PreprocessingLinearModel:
    original data preprocessing and testing
    input:
        x_df:feature frame
        pickle_name:model weight name
     return: 
        predicted value

In [21]:
def TestLinearModel(x_df,pickle_name='Linear_model.pkl'):
    x_df,_ = feature_clean(x_df, None, cleaning_x=True)
    x_df = data_prepare(x_df)
    input_feature1, output_feature1 = _1st_stacking_feature('EC')
    input_feature2, output_feature2 = _1st_stacking_feature('GFS')
    input_feature=input_feature1+input_feature2
    model=pickle.load(open(pickle_name, 'rb'))
    y_pred=model.predict(x_df[input_feature])
    plt.figure(figsize=(20,10))
    plt.plot(y_pred)
    plt.show()
    return y_pred

In [22]:
PreprocessingLinearModel(x_df,y_df[['power']])

89 (26901,)
       i.set X_basic.forecast_time  X_basic.horizon         X_basic.time  \
0          0   2018-07-01 00:00:00                0  2018-07-01 00:00:00   
1          0   2018-07-01 00:00:00                1  2018-07-01 01:00:00   
2          0   2018-07-01 00:00:00                2  2018-07-01 02:00:00   
3          0   2018-07-01 00:00:00                3  2018-07-01 03:00:00   
4          0   2018-07-01 00:00:00                4  2018-07-01 04:00:00   
...      ...                   ...              ...                  ...   
26871    548   2019-12-31 00:00:00               19  2019-12-31 19:00:00   
26872    548   2019-12-31 00:00:00               20  2019-12-31 20:00:00   
26873    548   2019-12-31 00:00:00               21  2019-12-31 21:00:00   
26874    548   2019-12-31 00:00:00               22  2019-12-31 22:00:00   
26875    548   2019-12-31 00:00:00               23  2019-12-31 23:00:00   

       X_basic.hour          EC.nwp_time  EC.dist     EC.ws       EC.wd  \


KeyError: 'EC.dist_ws'

In [23]:
y_ped=TestLinearModel(x_df)

89 (26901,)


KeyError: 'EC.dist_ws'